In [1]:
from videopipeline import Args, inference_by_imgs
from pathlib import Path

def load_prompts(prompt_file):
    f = open(prompt_file, 'r')
    prompt_list = []
    for idx, line in enumerate(f.readlines()):
        l = line.strip()
        if len(l) != 0:
            prompt_list.append(l)
        f.close()
    return prompt_list

/tmp2/b09902128/miniconda3/envs/videopipeline/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt_dir = Path('testprompts')
imgs = sorted(list(prompt_dir.glob("*.[jpJP][npNP][egEG]*")))
prompts = load_prompts(prompt_dir / 'test_prompts.txt')

args = Args()

In [3]:
inference_by_imgs(args, imgs, prompts)

AE working on z of shape (1, 4, 32, 32) = 4096 dimensions.
>>> model checkpoint loaded.
Inference with 32 frames


3it [01:53, 37.90s/it]
/tmp2/b09902128/miniconda3/envs/videopipeline/lib/python3.9/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
